In [1]:
import findspark
findspark.init()

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1 pyspark-shell'

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.streaming import *
from pyspark.sql.types import *
from pyspark.sql.functions import * 
import pyspark.sql.functions as f

In [4]:
spark = SparkSession.builder \
.master("local[4]") \
.appName("ReadKafka") \
.getOrCreate()

In [5]:
schema_simple_data = StructType(
[
    StructField("sirano", IntegerType(), True),
    StructField("isim", StringType(), True),
    StructField("yas", IntegerType(), True),
    StructField("meslek", StringType(), True),
    StructField("sehir", StringType(), True),
    StructField("aylik_gelir", DoubleType(), True)
]
)

In [6]:
df = spark \
.readStream \
.format("kafka") \
.option("kafka.bootstrap.servers","localhost:9092") \
.option("subscribe","topicDenemeCSV") \
.load()

In [7]:
df2=df.select(col("value").cast("string")) .alias("csv").select("csv.*")

In [8]:
df3=df2 \
      .selectExpr("split(value,',')[0] as sira_no" \
                 ,"split(value,',')[1] as isim" \
                 ,"split(value,',')[2] as yas" \
                 ,"split(value,',')[3] as meslek" \
                 ,"split(value,',')[4] as sehir" \
                 ,"split(value,',')[5] as aylik_gelir" 
                 
                 )

In [9]:
GroupBy = df3.groupBy("meslek") \
    .agg(f.avg("aylik_gelir").alias("ort_gelir")) \
    .sort(f.desc("ort_gelir"))

In [15]:
query = GroupBy.writeStream \
.format("console") \
.outputMode("complete") \
.trigger(processingTime='1 seconds') \
.start()

In [ ]:
query.awaitTermination()